In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('test_data_regno.csv', sep=';')

In [3]:
data.head()

,id,camera_id,time_check,regno_recognize,regno_ai,regno,result_code,symbols,symbol_scores,length_scores
0,1,844,00:00:00,У991КВ799,У991КВ799,У991КВ799,0,"[""У"", ""9"", ""9"", ""1"", ""К"", ""В"", ""7"", ""9"", ""9""]","[0.9999366998672485, 0.9999223947525024, 0.999...","[0.000000034831597162110484, 0.000000034835451..."
1,2,2516,00:00:00,О019УК177,О019УК177,О019УК177,0,"[""О"", ""0"", ""1"", ""9"", ""У"", ""К"", ""1"", ""7"", ""7""]","[0.9998993873596191, 0.999862790107727, 0.9999...","[0.0000000068477667980459955, 0.00000000683332..."
2,3,2476,00:00:00,Е175АК150,Е175АК150,Е175АК150,0,"[""Е"", ""1"", ""7"", ""5"", ""А"", ""К"", ""1"", ""5"", ""0""]","[0.9997256398200989, 0.9999269247055054, 0.999...","[0.000000002039653557162069, 0.000000002043387..."
3,4,1251,00:00:01,Т359ОН799,Т359ОН799,Т359ОН799,0,"[""Т"", ""3"", ""5"", ""9"", ""О"", ""Н"", ""7"", ""9"", ""9""]","[0.9998781681060791, 0.9998764991760254, 0.999...","[0.00000002880540606042814, 0.0000000289985120..."
4,5,2516,00:00:01,В521СУ750,В521СУ750,В521СУ750,0,"[""В"", ""5"", ""2"", ""1"", ""С"", ""У"", ""7"", ""5"", ""0""]","[0.999920129776001, 0.9999333620071411, 0.9999...","[0.000000009300508807541519, 0.000000009201486..."


In [4]:
dif_regno=data[data['regno_recognize']!=data['regno']]

In [5]:
#dif_regno.head()

In [6]:
bad_regno=data[data['result_code']==1]

In [7]:
#bad_regno.head()

In [8]:
bad_image=data[data['result_code']==4]

In [9]:
#bad_image.head()

In [10]:
fixed_regno=data[data['regno_recognize']!=data['regno']]

In [11]:
#fixed_regno.head()

In [12]:
not_corrected_regno=data[(data['regno_recognize']==data['regno']) & (data['regno_recognize']!=data['regno_ai'])]

In [13]:
#not_corrected_regno.head(10)

In [14]:
def IsChangedValue(x):
    if (x['regno_recognize']!=x['regno']):
        return 1
    else:
        return 0

In [15]:
y=data.apply(IsChangedValue, axis=1)

In [16]:
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [17]:
data['regnoChanged']=y

In [29]:
# получить отдельные оценки для всех знаков
# получить отдельные оценки для всех знаков, которые выбрали из второго алгоритма,
# получить отдельные оценки для всех знаков, которыми заменили отсутствующие символы,
# предсказанную длинну символов и метку о том, соотвествует ли предсказанная длина распознанной
def getDict(x, symbol_scores_array,length_scores_array, symbols_array):
    row=int(x['id'])-1
    row=int(row)
    temp_dict={}
    rec=x['regno_recognize']
    ai=x['regno_ai']
    final=x['regno']
    len1=len(rec)
    len2= len(ai)
    len3=len(final)
    #дополнить номер до итогового(regno), если длины разные, но одно начало.
    if len3 > len1 and len1>0:
        start=final.find(rec[0])
        if start==0:
            rec=rec.ljust(len3, '_')
            len1=len3
        else:
            if start>0:
                rec=rec.rjust(len3, '_')
                len1=len3
    min_len=min(len2, len3)
    
    #количество 'regno_recognize' не совпавших с 'regno_ai'   
    nonai_count=0
    empty_count=0
    # пройти по всем символам, собрать оценки для замененных символов и отдельно для замененных пустых
    for i in range(min_len):
        if i<len1:
            rec_val=rec[i]
        else:
            rec_val='_'
        if rec_val != final[i] and symbols_array[row][i]==ai[i]:
            if ai[i]==final[i]:
                temp_dict['selected_symb_score' + str(i)]=symbol_scores_array[row][i]
                if rec_val=='_' or rec_val.strip()=='':
                    temp_dict['empty_symb_score' + str(i)]=symbol_scores_array[row][i]
        if rec_val != ai[i]:
            nonai_count=nonai_count+1
        temp_dict['symb_score' + str(i)]=symbol_scores_array[row][i]
        if rec_val =='_' or rec_val.strip()=='':
            empty_count=empty_count+1
    temp_dict['length_predicted']=np.argmax(length_scores_array[row])
    
    # чтобы учесть количество отличий 
    for i in range(min_len, min(len2, len1)):
        if rec[i] != ai[i]:
            nonai_count=nonai_count+1
    nonai_count=np.abs(len2-len1)+nonai_count
    
    temp_dict['nonai_count']=nonai_count
    if len1-empty_count==temp_dict['length_predicted']:
        temp_dict['is_length_equal']=1
    else:
        temp_dict['is_length_equal']=0
    return  temp_dict

In [30]:
# парсинг оценок
def GetScoresArray(X_dt):
    symbols_array=[list(map(str.strip, symbol[2:-2].replace('"','').split(','))) for symbol in  X_dt['symbols']]
    symbol_scores_array=[list(map(float, symbol[1:-1].split(','))) for symbol in X_dt['symbol_scores']]
    length_scores_array=[list(map(float, score[1:-1].split(','))) for score in  X_dt['length_scores']]
    return symbols_array,symbol_scores_array, length_scores_array

In [31]:
symbols_array,symbol_scores_array, length_scores_array = GetScoresArray(data)

In [32]:
# получить новый фрейм с оценками
def GetNewFeatures(X_dt):
    dif_scores=pd.DataFrame(list(X_dt.apply(getDict, axis=1, 
                args=[symbol_scores_array,length_scores_array, symbols_array])))
    dif_scores.fillna(0, inplace=True)
    return dif_scores

In [33]:
dif_scores=GetNewFeatures(data)

In [34]:
dif_scores.tail(5)

,empty_symb_score0,empty_symb_score1,empty_symb_score2,empty_symb_score3,empty_symb_score4,empty_symb_score5,empty_symb_score6,empty_symb_score7,empty_symb_score8,is_length_equal,...,selected_symb_score8,symb_score0,symb_score1,symb_score2,symb_score3,symb_score4,symb_score5,symb_score6,symb_score7,symb_score8
194209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.999865,0.999917,0.999926,0.999944,0.999880,0.998932,0.999953,0.999848,0.999960
194210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,0.999926,0.999952,0.999959,0.999929,0.999944,0.999907,0.999740,0.999835,0.000000
194211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,0.999947,0.999940,0.999933,0.999950,0.999941,0.999775,0.999982,0.999961,0.999777
194212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,0.999943,0.999817,0.999786,0.999802,0.999993,0.999968,0.999463,0.999357,0.000000
194213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,...,0.0,0.999854,0.999925,0.999888,0.999929,0.999908,0.999886,0.999984,0.999968,0.999957


In [35]:
data.tail(5)

,id,camera_id,time_check,regno_recognize,regno_ai,regno,result_code,symbols,symbol_scores,length_scores,regnoChanged
194209,194210,5497,23:59:58,В035Т_197,В035ТС197,В035Т_197,1,"[""В"", ""0"", ""3"", ""5"", ""Т"", ""С"", ""1"", ""9"", ""7""]","[0.9998652935028076, 0.9999170303344727, 0.999...","[0.000000012212018951629489, 0.000000012157821...",0
194210,194211,6320,23:59:58,А974КА67,А974КА67,А974КА67,2,"[""А"", ""9"", ""7"", ""4"", ""К"", ""А"", ""6"", ""7""]","[0.9999264478683472, 0.9999524354934692, 0.999...","[0.0000000026609143777278632, 0.00000000267139...",0
194211,194212,2659,23:59:58,Х037НН777,Х037НН777,Х037НН777,0,"[""Х"", ""0"", ""3"", ""7"", ""Н"", ""Н"", ""7"", ""7"", ""7""]","[0.9999473094940186, 0.9999399185180664, 0.999...","[0.0000000758142704171405, 0.00000007637027721...",0
194212,194213,839,23:59:59,С110ОС66,С110ОС69,С110ОС69,5,"[""С"", ""1"", ""1"", ""0"", ""О"", ""С"", ""6"", ""9""]","[0.9999434947967529, 0.999817430973053, 0.9997...","[0.000000009674453238517344, 0.000000009729264...",1
194213,194214,1251,23:59:59,Х772НР777,Х772НР777,Х772НР777,0,"[""Х"", ""7"", ""7"", ""2"", ""Н"", ""Р"", ""7"", ""7"", ""7""]","[0.9998537302017212, 0.9999246597290039, 0.999...","[0.00000003661622827166866, 0.0000000367221844...",0


In [36]:
selected_columns=['camera_id','time_check','regno_recognize','regno_ai','regno','result_code','regnoChanged','symbol_scores']

In [37]:
new_columns=dif_scores.columns

In [38]:
#X=pd.concat([data[selected_columns], dif_scores], axis=1)
# объединим фреймы для наглядности
X=data
X[new_columns]=dif_scores

In [39]:
X.head(11)

,id,camera_id,time_check,regno_recognize,regno_ai,regno,result_code,symbols,symbol_scores,length_scores,...,selected_symb_score8,symb_score0,symb_score1,symb_score2,symb_score3,symb_score4,symb_score5,symb_score6,symb_score7,symb_score8
0,1,844,00:00:00,У991КВ799,У991КВ799,У991КВ799,0,"[""У"", ""9"", ""9"", ""1"", ""К"", ""В"", ""7"", ""9"", ""9""]","[0.9999366998672485, 0.9999223947525024, 0.999...","[0.000000034831597162110484, 0.000000034835451...",...,0.0,0.999937,0.999922,0.999883,0.999906,0.999920,0.999878,0.999733,0.999527,0.999081
1,2,2516,00:00:00,О019УК177,О019УК177,О019УК177,0,"[""О"", ""0"", ""1"", ""9"", ""У"", ""К"", ""1"", ""7"", ""7""]","[0.9998993873596191, 0.999862790107727, 0.9999...","[0.0000000068477667980459955, 0.00000000683332...",...,0.0,0.999899,0.999863,0.999929,0.999959,0.999959,0.999864,0.999864,0.999885,0.999973
2,3,2476,00:00:00,Е175АК150,Е175АК150,Е175АК150,0,"[""Е"", ""1"", ""7"", ""5"", ""А"", ""К"", ""1"", ""5"", ""0""]","[0.9997256398200989, 0.9999269247055054, 0.999...","[0.000000002039653557162069, 0.000000002043387...",...,0.0,0.999726,0.999927,0.999964,0.999946,0.999908,0.999943,0.999868,0.999798,0.999853
3,4,1251,00:00:01,Т359ОН799,Т359ОН799,Т359ОН799,0,"[""Т"", ""3"", ""5"", ""9"", ""О"", ""Н"", ""7"", ""9"", ""9""]","[0.9998781681060791, 0.9998764991760254, 0.999...","[0.00000002880540606042814, 0.0000000289985120...",...,0.0,0.999878,0.999876,0.999909,0.999907,0.999792,0.999662,0.999265,0.999297,0.999295
4,5,2516,00:00:01,В521СУ750,В521СУ750,В521СУ750,0,"[""В"", ""5"", ""2"", ""1"", ""С"", ""У"", ""7"", ""5"", ""0""]","[0.999920129776001, 0.9999333620071411, 0.9999...","[0.000000009300508807541519, 0.000000009201486...",...,0.0,0.999920,0.999933,0.999909,0.999906,0.999577,0.999943,0.999951,0.999914,0.999971
5,6,891,00:00:01,АУ62199,АУ62199,АУ62199,0,"[""А"", ""У"", ""6"", ""2"", ""1"", ""9"", ""9""]","[0.9999922513961792, 0.9991457462310791, 0.999...","[0.000000002017199740578235, 0.000000002019802...",...,0.0,0.999992,0.999146,0.999989,0.999925,0.998611,0.999779,0.999985,0.000000,0.000000
6,7,1840,00:00:02,Е711ТМ777,Е711ТМ777,Е711ТМ777,4,"[""Е"", ""7"", ""1"", ""1"", ""Т"", ""М"", ""7"", ""7"", ""7""]","[0.9998533725738525, 0.9998818635940552, 0.999...","[0.000000006809274477603822, 0.000000006806080...",...,0.0,0.999853,0.999882,0.999897,0.999959,0.999927,0.999937,0.999993,0.999998,0.999995
7,8,6547,00:00:02,М199ХУ777,М199ХУ777,М199ХУ777,0,"[""М"", ""1"", ""9"", ""9"", ""Х"", ""У"", ""7"", ""7"", ""7""]","[0.9997625946998596, 0.9998775720596313, 0.999...","[0.0000000467794514236175, 0.00000004675741749...",...,0.0,0.999763,0.999878,0.999821,0.999871,0.999811,0.999829,0.999942,0.999965,0.999964
8,9,752,00:00:04,У941ЕМ799,У941ЕМ799,У941ЕМ799,5,"[""У"", ""9"", ""4"", ""1"", ""Е"", ""М"", ""7"", ""9"", ""9""]","[0.9996079802513123, 0.9998847246170044, 0.999...","[0.00000001306333441419838, 0.0000000130716086...",...,0.0,0.999608,0.999885,0.999907,0.999960,0.999954,0.999906,0.999760,0.999612,0.999796
9,10,1296,00:00:05,Х304УМ197,Х304УМ197,Х304УМ197,0,"[""Х"", ""3"", ""0"", ""4"", ""У"", ""М"", ""1"", ""9"", ""7""]","[0.9999352693557739, 0.9998856782913208, 0.999...","[0.000000011356738660595056, 0.000000011327728...",...,0.0,0.999935,0.999886,0.999866,0.999914,0.999922,0.999898,0.999975,0.999959,0.999959


In [40]:
#просмотр тех данных, где пользователь поменял номер
X_changed=X[X['regnoChanged']==1]

In [41]:
#просмотр корректынх данных, где пользователь поменял номер
X_changed_correct=X[X['regnoChanged']==1  & (X['result_code']==0)]

In [42]:
#минимальные оценки для заданых колонок,
def PrintMinscores(x_dt, columns_prefix, print_values):
    for col in x_dt.columns.values:
        if str(col).find(columns_prefix)==0:
            values=x_dt[(x_dt[col]>0) ]
            print(col, np.min(list(values[col])))
            if print_values==True:
                print (values.iloc[np.argmin(list(values[col]))])

In [43]:
#средние оценки для заданых колонок,
def PrintAvgScores(x_dt, columns_prefix):
    for col in x_dt.columns.values:
        if str(col).find(columns_prefix)==0:
            values=x_dt[(x_dt[col]>0) ]
            print(col,"avg:", np.average(list(values[col])))
            #частоты еще посмотрим
            t=list(x_dt[x_dt[col]>0][col].apply(lambda x: int(x*100)))
            counts = np.bincount(t)
            print (col, "frequent:", np.argmax(counts))

In [44]:
PrintMinscores(X_changed_correct,'selected_sym', True)

selected_symb_score0 0.7681621313095093
id                                                                  49198
camera_id                                                             719
time_check                                                       08:21:12
regno_recognize                                                 _655А_178
regno_ai                                                        Н655АУ178
regno                                                           Н655АХ178
result_code                                                             0
symbols                     ["Н", "6", "5", "5", "А", "У", "1", "7", "8"]
symbol_scores           [0.7681621313095093, 0.9991084933280945, 0.999...
length_scores           [0.000000009418846147468685, 0.000000009371929...
regnoChanged                                                            1
empty_symb_score0                                                0.768162
empty_symb_score1                                                       

при просмотре данных, можно увидеть

    regno_recognize                                                  О642ЕС07
    regno_ai                                                         О642ЕС15
    regno                                                            О642ЕС05

При этом 

    symb_score6                                                      0.419012
    symb_score7                                                      0.398907


1) верроятности малы, и они не позволяют создать правило для коррекции по вероятности. Т.е. выбирать второй алгоритм, 
если у него вероятность символа >0.8

2) да и выбор по вероятности может испортить данные, поэтому лучше создать правило для отсутсвующих символов.

In [45]:
PrintMinscores(X_changed_correct,'empty_symb', False)

empty_symb_score0 0.7681621313095093
empty_symb_score1 0.4116600751876831
empty_symb_score2 0.9706540703773499
empty_symb_score3 0.6913644075393677
empty_symb_score4 0.5640883445739746
empty_symb_score5 0.5322401523590088
empty_symb_score6 0.47616010904312134
empty_symb_score7 0.5535935163497925
empty_symb_score8 0.4956435263156891


In [46]:
PrintAvgScores(X_changed_correct,'empty_symb')

empty_symb_score0 avg: 0.9988732549163314
empty_symb_score0 frequent: 99
empty_symb_score1 avg: 0.9902088947074358
empty_symb_score1 frequent: 99
empty_symb_score2 avg: 0.999016918738683
empty_symb_score2 frequent: 99
empty_symb_score3 avg: 0.9953684193500574
empty_symb_score3 frequent: 99
empty_symb_score4 avg: 0.990368912473393
empty_symb_score4 frequent: 99
empty_symb_score5 avg: 0.9964846699151269
empty_symb_score5 frequent: 99
empty_symb_score6 avg: 0.9967810788973547
empty_symb_score6 frequent: 99
empty_symb_score7 avg: 0.9977778477448944
empty_symb_score7 frequent: 99
empty_symb_score8 avg: 0.9994313773663699
empty_symb_score8 frequent: 99


Из полученного выше, можно попробовать вывести правило:

    при отсутсвии символа, берем соответсвующий символ из символов второго алгоритма, если оценка символа > 0.8(к примеру)

In [47]:
# Посмотрим, на сколько это верно для всего набора данных ( не только для корректных данных)
PrintMinscores(X_changed,'empty_symb', False)

empty_symb_score0 0.21112407743930817
empty_symb_score1 0.2402435690164566
empty_symb_score2 0.4626888334751129
empty_symb_score3 0.3969961106777191
empty_symb_score4 0.30647164583206177
empty_symb_score5 0.30388888716697693
empty_symb_score6 0.1715165376663208
empty_symb_score7 0.24024417996406555
empty_symb_score8 0.38547855615615845


In [48]:
PrintAvgScores(X_changed,'empty_symb')

empty_symb_score0 avg: 0.9484814749165693
empty_symb_score0 frequent: 99
empty_symb_score1 avg: 0.9485032727390947
empty_symb_score1 frequent: 99
empty_symb_score2 avg: 0.9590295497822551
empty_symb_score2 frequent: 99
empty_symb_score3 avg: 0.9797963509741037
empty_symb_score3 frequent: 99
empty_symb_score4 avg: 0.9859942693787792
empty_symb_score4 frequent: 99
empty_symb_score5 avg: 0.9857205996650675
empty_symb_score5 frequent: 99
empty_symb_score6 avg: 0.974277167632107
empty_symb_score6 frequent: 99
empty_symb_score7 avg: 0.9966611296581708
empty_symb_score7 frequent: 99
empty_symb_score8 avg: 0.9990049135256296
empty_symb_score8 frequent: 99


В данном случае минимальные вероятности выбранных символов (для замен пустых) малы.

In [49]:
#попробуем только те, у которых лишь менее трех символов отличны от символов второго алгоритма
X_nonai=X_changed[X_changed['nonai_count']<4]

In [50]:
PrintMinscores(X_nonai,'empty_symb',False)

empty_symb_score0 0.7681621313095093
empty_symb_score1 0.4116600751876831
empty_symb_score2 0.9706540703773499
empty_symb_score3 0.6913644075393677
empty_symb_score4 0.5640883445739746
empty_symb_score5 0.3982970714569092
empty_symb_score6 0.3714984655380249
empty_symb_score7 0.5535935163497925
empty_symb_score8 0.38547855615615845


In [51]:
PrintAvgScores(X_nonai,'empty_symb')

empty_symb_score0 avg: 0.9990217891808982
empty_symb_score0 frequent: 99
empty_symb_score1 avg: 0.9787897997432284
empty_symb_score1 frequent: 99
empty_symb_score2 avg: 0.9989124233906086
empty_symb_score2 frequent: 99
empty_symb_score3 avg: 0.9935578006964464
empty_symb_score3 frequent: 99
empty_symb_score4 avg: 0.9954327076544494
empty_symb_score4 frequent: 99
empty_symb_score5 avg: 0.9926501164032567
empty_symb_score5 frequent: 99
empty_symb_score6 avg: 0.9944883388150889
empty_symb_score6 frequent: 99
empty_symb_score7 avg: 0.9968752248280522
empty_symb_score7 frequent: 99
empty_symb_score8 avg: 0.9991642695678324
empty_symb_score8 frequent: 99


In [52]:
counts = np.bincount(list(X_changed['nonai_count']))
print (counts)
print (np.argmin(counts))

[  51 6873 1959  671  394  280  284  268  201   68   42   19    1]
12


Вывод: чаще всего изменения вносились, когда разница в символах алгоритма была 1,2 и 3.

In [53]:
counts = np.bincount(list(X['nonai_count']))
print (counts)
print (np.argmin(counts))

[166724  11558   4211   2976   2861   2247   1617   1144    637    177
     42     19      1]
12


In [54]:
correct_cameras=X[X['regnoChanged']==0  & (X['result_code']==0)]['camera_id']

In [55]:
incorrect_cameras=X[X['regnoChanged']==1  & (X['result_code']!=0)]['camera_id']

In [56]:
# зависит ли от камеры
len(np.unique(correct_cameras)), len(np.unique(incorrect_cameras))

(971, 959)

In [57]:
import collections
import operator
correct_camera_counts=collections.Counter(correct_cameras)
correct_camera_counts.most_common(10)

[(6090, 4848),
 (2516, 2597),
 (781, 2427),
 (812, 2223),
 (6445, 2192),
 (740, 2051),
 (1977, 1632),
 (8599, 1507),
 (7096, 1427),
 (5065, 1380)]

In [58]:
incorrect_camera_counts=collections.Counter(incorrect_cameras)
incorrect_camera_counts.most_common(10)

[(6090, 4205),
 (2516, 2354),
 (781, 2176),
 (812, 2023),
 (740, 1713),
 (6445, 1658),
 (8599, 1399),
 (1977, 1373),
 (7096, 1276),
 (5065, 1244)]

скорее всего камеры исправны

In [59]:
from datetime import datetime
def GetHour(s): 
    return datetime.strptime(s, '%H:%M:%S').hour

In [60]:
correct_hour=X[X['regnoChanged']==0  & (X['result_code']==0)]['time_check'].apply(GetHour)

In [61]:
# можно ли выявить зависимоть качества от времени суток
correct_hour_counts=collections.Counter(correct_hour)
correct_hour_counts.most_common(10)

[(15, 10753),
 (14, 10264),
 (16, 10215),
 (13, 10112),
 (12, 10003),
 (11, 9534),
 (10, 9146),
 (17, 9079),
 (18, 8659),
 (9, 8634)]

In [62]:
incorrect_hour=X[X['regnoChanged']==1  & (X['result_code']!=0)]['time_check'].apply(GetHour)

In [63]:
incorrect_hour_counts=collections.Counter(incorrect_hour)
incorrect_hour_counts.most_common(10)
#полученные данные не особо информативны

[(15, 7920),
 (16, 7567),
 (14, 7485),
 (13, 7447),
 (12, 7369),
 (11, 7053),
 (17, 6802),
 (10, 6675),
 (18, 6482),
 (9, 6187)]

# В итоге получаю правило:

   ### Замену производить для отсутствующих символов, если выполнены все условия: 
    
    1) соответствующий символ regno_ai имеет вероятность > заданного значения min_valid_score
    ( которое в итоге = 0,8 );
    2) количество различий в символах 'regno_recognize' с 'regno_ai'должно быть меньше max_dif_count. (Должно увеличить точность) 
    3) в 'regno_recognize' нет непустых символов, которые не совпадают с соответствующими в 'regno_ai'  

    

In [64]:
# дополнить строку до нужной длины
def fillValue(start,val,len1):
    if start<0:
        return val
    if start==0:
        return val.ljust(len1, '_')
    if start>0:
        return val.rjust(len1, '_')

In [65]:
# алгоритм замены
def getNewVals(x, symbol_scores_array,length_scores_array, symbols_array, min_valid_score, max_dif_count):
    row=int(x['id'])-1
    row=int(row)
    temp_dict={}
    
    temp_dict['regnoChanged'] = x['regnoChanged']
    temp_dict['result_code']=x['result_code']
    
    rec=x['regno_recognize']
    ai=x['regno_ai']
    final=x['regno']
      
    
    len1 =len(rec)
    len2 = len(ai)
    len3 = len(final)
    if len1==0 or len2==0:
        return  temp_dict
    
    if rec==ai:     
        temp_dict['new_changed']=0  
        return temp_dict
    
     # дополнить меньшую из строк, если длины разные, но одно начало. 
    if len2 > len1:
        start=ai.find(rec[0])
        rec=fillValue(start, rec, len2)
        len1=len(rec)
    # количество различий в символах двух алгоритмов    
    nonai_count=0
    # количество пропущенных
    missing_count=0
    # новый номер
    new_reg=''
    temp_dict['has_missing_val']=0
   
    # количество симовлов у которых symbol_scores_array[row][i] < min_valid_score, но равных соответствующим сиволам из regno.
    equaltoai_count=0
    for i in range(len2):
        if i<len1:
            rec_val=rec[i]
        else:
            rec_val='_'       
      
        
        if rec_val != ai[i]:
            # подсчет количества различий в символах двух алгоритмв.  
            nonai_count=nonai_count+1
        if rec_val!='_' and rec_val.strip()!='':
            # сбор нового номера.
            new_reg=new_reg+rec_val
        else:
            # если в 'regno_recognize' отсутсвует символ заменяем на символ 'regno_ai'.
            # при валидной вероятности символа.
            if symbol_scores_array[row][i] > min_valid_score:
                new_reg=new_reg+ai[i]
            else:
                # если вероятнрость мала, но добавленный символ попал в итоговы номер, посчитаем его, 
                # это нужно для подсчета 'false_negative'.
                if i <len3 and ai[i]==final[i]:
                      equaltoai_count= equaltoai_count+1
                new_reg=new_reg+rec_val
            # подсчет количества пустых.
            missing_count=missing_count+1
            temp_dict['has_missing_val']=1
    # чтобы отобразить количество отличий в символах  
    temp_dict['nonai_count']=nonai_count
    # если отличаются только пустые смиволы и количество отличий < max_dif_count, производим замену.
    if nonai_count< max_dif_count and new_reg!=rec and missing_count==nonai_count and missing_count>0:     
        temp_dict['new_regno']=new_reg
        # пометка о произведенной замене.
        temp_dict['new_changed']=1
        if new_reg==final:
            temp_dict['true_positive']=1
        else:
            temp_dict['false_positive']=1
    else: 
        if new_reg!=rec:
            if equaltoai_count == missing_count:
                temp_dict['false_negative']=1
            else:
                temp_dict['true_negative']=1
        temp_dict['new_changed']=0
    
    return  temp_dict

In [66]:
# получим номера по определенному правилу
def GetNewFarme(X_dt,  min_valid_score, max_dif_count): 
    newFarme=pd.DataFrame(list(X_dt.apply(getNewVals, axis=1, 
                args=[symbol_scores_array,length_scores_array, symbols_array,min_valid_score, max_dif_count])))
     
    newFarme.fillna(0, inplace=True)
    return newFarme

In [67]:
addcolumns=[ 'regno', 'regno_recognize','regno_ai']

In [68]:
X_new=GetNewFarme(data,0.7,3)

In [69]:
X_new[addcolumns]=data[addcolumns]

In [70]:
X_new.tail()

,false_negative,false_positive,has_missing_val,new_changed,new_regno,nonai_count,regnoChanged,result_code,true_negative,true_positive,regno,regno_recognize,regno_ai
194209,0.0,1.0,1.0,1,В035ТС197,1.0,0,1,0.0,0.0,В035Т_197,В035Т_197,В035ТС197
194210,0.0,0.0,0.0,0,0,0.0,0,2,0.0,0.0,А974КА67,А974КА67,А974КА67
194211,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,Х037НН777,Х037НН777,Х037НН777
194212,0.0,0.0,0.0,0,0,1.0,1,5,0.0,0.0,С110ОС69,С110ОС66,С110ОС69
194213,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,Х772НР777,Х772НР777,Х772НР777


In [71]:
#опять, смотрим только измененные пользователем данные
X_new_changed=X_new[X_new['regnoChanged']==1]

In [72]:
X_new_changed.head()

,false_negative,false_positive,has_missing_val,new_changed,new_regno,nonai_count,regnoChanged,result_code,true_negative,true_positive,regno,regno_recognize,regno_ai
29,0.0,0.0,0.0,0,0,1.0,1,0,0.0,0.0,А287АМ777,А787АМ777,А287АМ777
52,0.0,0.0,1.0,1,М039НК799,1.0,1,2,0.0,1.0,М039НК799,М039НК79_,М039НК799
53,0.0,0.0,1.0,0,0,2.0,1,0,1.0,0.0,У118УС750,У118УС76,У118УС750
58,0.0,0.0,0.0,0,0,1.0,1,2,0.0,0.0,А576ТВ198,А576ТВ190,А576ТВ199
74,0.0,0.0,1.0,1,О575НО777,1.0,1,0,0.0,1.0,О575НО777,О575НО77,О575НО777


In [73]:
#  подсчет ошибки f1
def getError(X_new_dt, print_errors=False):
    tp=np.sum(X_new_dt['true_positive'])
    fp=np.sum(X_new_dt['false_positive'])
    fn=np.sum(X_new_dt['false_negative'])
    if tp==0 and fp ==0:
        return 0
    precision=tp/(tp+fp)   
    recall=tp/(tp+fn)  
    
   #значения  которые не стали заменять
    missing_count = np.sum(X_new_dt['has_missing_val'])
    not_changed=missing_count-tp-fp
    f1=2*precision*recall/(precision+recall)
    if print_errors == True:
        print('missing_count ', missing_count)
        print('not_changed ', not_changed)
        print('recall ',recall)
        print('precision ', precision)        
        print('f1 ', f1)
    return f1

In [74]:
getError(X_new_changed,True)

missing_count  7642.0
not_changed  3072.0
recall  0.963135593220339
precision  0.9947483588621444
f1  0.978686759956943


0.978686759956943

In [75]:
X_new_correct=X_new_changed[X_new_changed['result_code']==0]

In [76]:
getError(X_new_correct,True)

missing_count  5828.0
not_changed  1920.0
recall  0.9850821744627054
precision  0.9969293756397134
f1  0.9909703675441943


0.9909703675441943

### Вывод: 
    больше половины номеров, в которых пропущен хоть один символ, можно с минимальной ошибкой заменить на символы, 
    полученные заданным правилом.

In [77]:
X_new_changed[(X_new_changed['true_positive']==0) & (X_new_changed['new_changed']==1)]

,false_negative,false_positive,has_missing_val,new_changed,new_regno,nonai_count,regnoChanged,result_code,true_negative,true_positive,regno,regno_recognize,regno_ai
8234,0.0,1.0,1.0,1,А712ТЕ197,1.0,1,5,0.0,0.0,Р712ТЕ197,_712ТЕ197,А712ТЕ197
9463,0.0,1.0,1.0,1,УУ41899,1.0,1,5,0.0,0.0,У941899,У_41899,УУ41899
10168,0.0,1.0,1.0,1,В324АТ190,1.0,1,0,0.0,0.0,В324АТ198,В324АТ19,В324АТ190
12611,0.0,1.0,1.0,1,С694ОЕ799,1.0,1,0,0.0,0.0,С674ОЕ799,С6_4ОЕ799,С694ОЕ799
14970,0.0,1.0,1.0,1,Р848РО777,1.0,1,0,0.0,0.0,Р848РС777,Р848Р_777,Р848РО777
18762,0.0,1.0,1.0,1,В419ЕО190,1.0,1,0,0.0,0.0,В419ЕО198,В419ЕО19,В419ЕО190
27718,0.0,1.0,1.0,1,Е006СР190,1.0,1,0,0.0,0.0,Е006ОР190,Е006СР19,Е006СР190
49197,0.0,1.0,1.0,1,Н655А_178,2.0,1,0,0.0,0.0,Н655АХ178,_655А_178,Н655АУ178
49936,0.0,1.0,1.0,1,В112ХТ19,1.0,1,5,0.0,0.0,В112ХТ,В112ХТ_9,В112ХТ19
55049,0.0,1.0,1.0,1,В446ХХ197,1.0,1,0,0.0,0.0,В446ХХ199,В446ХХ19,В446ХХ197


In [78]:
# Подбор параметра
#min_valid_score, max_dif_count
parateters_toselect=[(0.5,3),(0.7, 3), (0.8,3), (0.8,4), (0.9,3), (0.9,4)]
scores=[]
for vals in parateters_toselect:  
    frame=GetNewFarme(data,vals[0],vals[1]) 
    frame_=frame[frame['regnoChanged']==1]
    print(vals[0]," ", vals[1])
    scores.append(getError(frame_,True))


0.5   3
missing_count  7642.0
not_changed  3064.0
recall  0.9643916913946587
precision  0.9938837920489296
f1  0.9789156626506025
0.7   3
missing_count  7642.0
not_changed  3072.0
recall  0.963135593220339
precision  0.9947483588621444
f1  0.978686759956943
0.8   3
missing_count  7642.0
not_changed  3078.0
recall  0.962507943232366
precision  0.9956178790534619
f1  0.9787829833064082
0.8   4
missing_count  7642.0
not_changed  2964.0
recall  0.9633540372670808
precision  0.9946558358272766
f1  0.9787547328565419
0.9   3
missing_count  7642.0
not_changed  3087.0
recall  0.9612534406097819
precision  0.9967069154774972
f1  0.9786591937917655
0.9   4
missing_count  7642.0
not_changed  2973.0
recall  0.9621196439660525
precision  0.9955022488755623
f1  0.9785263157894737


In [79]:
print (parateters_toselect[np.argmax(scores)])

(0.5, 3)


Наиболее оптимальный вариант 

0.8, 4

    f1  0.9787547328565419
    missing_count  7642.0
    not_changed  2964.0
    recall  0.9633540372670808
    precision  0.9946558358272766
    
Тут точность хорошая, и заменнили больше данных.

## Вывод: 
    Описанное правило помогает подставлять наиболее вероятные символы на места пропущенных символов.
    При этом не происходит затирания корректных символов, а больше половины номеров, в которых пропущен хоть один символ,
    можно с минимальной ошибкой заменить на символы второго алгоритма,
    что заметно сократит объем просматриваемых оператором данных.
